In [1]:
# Chargement du dataset augmenté dans un DataFrame Pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset # Pour Hugging Face datasets
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

INPUT_AUGMENTED_CSV_PATH = "corpus_augmente_paraphrases.csv"
df_augmented = pd.read_csv(INPUT_AUGMENTED_CSV_PATH)

if 'label_id' not in df_augmented.columns:
    unique_labels = df_augmented['catégorie'].unique()
    label_to_id = {label: i for i, label in enumerate(unique_labels)}
    df_augmented['label_id'] = df_augmented['catégorie'].map(label_to_id)
else:
    unique_labels = df_augmented['catégorie'].unique()
    label_to_id = {label: df_augmented[df_augmented['catégorie'] == label]['label_id'].iloc[0] for label in unique_labels}


print(f"Dataset chargé. Nombre d'articles : {len(df_augmented)}")
print(f"Mapping des labels : {label_to_id}")

Dataset chargé. Nombre d'articles : 80
Mapping des labels : {'culture': 0, 'sport': 1}


In [2]:
# Séparation des données en entraînement (80%), validation (10%) et test (10%)

train_df, temp_df = train_test_split(df_augmented, test_size=0.2, random_state=42, stratify=df_augmented['label_id'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label_id']) # 0.5 de 20% = 10%

print(f"Taille du jeu d'entraînement : {len(train_df)}")
print(f"Taille du jeu de validation : {len(val_df)}")
print(f"Taille du jeu de test : {len(test_df)}")

# Conversion les DataFrames en objets Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

Taille du jeu d'entraînement : 64
Taille du jeu de validation : 8
Taille du jeu de test : 8


In [3]:
# Tokenisation

MODEL_NAME = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["article"], padding="max_length", truncation=True, max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["article", "__index_level_0__"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["article", "__index_level_0__"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["article", "__index_level_0__"])

# Colonne 'label_id' renommée en 'labels' pour que le Trainer la reconnaisse
tokenized_train_dataset = tokenized_train_dataset.rename_column("label_id", "labels")
tokenized_val_dataset = tokenized_val_dataset.rename_column("label_id", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("label_id", "labels")

# Définition du format des datasets en PyTorch tensors
tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [4]:
# Entrainement du modèle de classification

# Chargement du modèle
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_labels))
# Déplacer le modèle vers le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modèle prêt sur : {device}")

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle prêt sur : cpu


In [5]:
# Métriques d'évaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary') # 'binary' pour 2 classes
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
)

/Users/catalinciomne/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer, # Passer le tokenizer pour les logs et la sauvegarde
)

print("\nDébut de l'entraînement du modèle...")
trainer.train()
print("Entraînement terminé.")

/var/folders/rx/bdq739r550xgs8_lp5tqqn4m0000gn/T/ipykernel_1493/546676335.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Début de l'entraînement du modèle...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.692417,0.500000,0.666667,0.500000,1.000000
2,No log,0.691955,0.500000,0.666667,0.500000,1.000000
3,No log,0.691183,0.500000,0.666667,0.500000,1.000000
4,No log,0.689995,0.500000,0.666667,0.500000,1.000000
5,No log,0.688406,0.500000,0.666667,0.500000,1.000000


Entraînement terminé.


In [8]:
# Evaluation

print("\nÉvaluation du modèle sur l'ensemble de test...")
results = trainer.evaluate(tokenized_test_dataset)
print(f"Résultats d'évaluation finaux sur le jeu de test : {results}")


Évaluation du modèle sur l'ensemble de test...


Résultats d'évaluation finaux sur le jeu de test : {'eval_loss': 0.6911566257476807, 'eval_accuracy': 0.5, 'eval_f1': 0.6666666666666666, 'eval_precision': 0.5, 'eval_recall': 1.0, 'eval_runtime': 2.1888, 'eval_samples_per_second': 3.655, 'eval_steps_per_second': 0.457, 'epoch': 5.0}
